In [1]:
library(RPostgreSQL)
library(tableone)
library(tidyverse)

Loading required package: DBI
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [4]:
df <- dbGetQuery(con, "select * from serum_echo_diff;")
head(df)

hadm_id,creatinine_diff,lactate_diff
159895,NA,NA
169700,NA,NA
183493,NA,NA
103722,NA,NA
170515,-0.5,NA
134244,NA,NA


In [5]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE

In [6]:
df_pivot <- df %>% gather("key", "value", -hadm_id)
head(df_pivot)

hadm_id,key,value
159895,creatinine_diff,NA
169700,creatinine_diff,NA
183493,creatinine_diff,NA
103722,creatinine_diff,NA
170515,creatinine_diff,-0.5
134244,creatinine_diff,NA


In [7]:
tab <- df_pivot %>%
pull(key) %>%
unique %>%
map(function(label) {
    x <- df_pivot %>% filter(key == label) %>% pull(value)
    tt <- t.test(x)
    m <- mean(na.omit(x))
    s <- sd(na.omit(x))
    lo <- tt$conf.int[1]
    hi <- tt$conf.int[2]
    p_val <- tt$p.value
    data.frame(covariate = label, m = m, s = s, lo = lo, hi = hi, p_value = p_val)
}) %>%
data.table::rbindlist() %>%
mutate(estimate = sprintf("%.2f (+/- %.2f)", m, s)) %>%
mutate(ci = sprintf("%.2f - %.2f", lo, hi)) %>%
mutate(p_value = round(p_value, 4)) %>%
select(covariate, estimate, ci, p_value)

tab

covariate,estimate,ci,p_value
creatinine_diff,-0.17 (+/- 1.17),-0.30 - -0.04,0.008
lactate_diff,-1.24 (+/- 2.50),-1.66 - -0.82,0.000


In [8]:
data.table::fwrite(tab, file.path(data_dir, "serum_echo_diff.csv"))